# MNIST with PyTorch


## some examples
- [Convolution arithmetic](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md)
- [Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
- [Basic MNIST example](https://github.com/pytorch/examples/tree/main/mnist)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch , log_interval, dry_run):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
batch_size = 64
test_batch_size = 1000
epochs = 2
lr = 1.0
gamma = 0.7
seed = 1
log_interval = 10
save_model = False
dry_run = False

torch.manual_seed(seed)
device = torch.device("cpu")

# Data loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('../data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size)

# Model setup
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval, dry_run)
    test(model, device, test_loader)
    scheduler.step()

if save_model:
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.326473
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.377823
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.835124
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.623733
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.439014
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.293272
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.168829
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.625972
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.180076
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.228200
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.285825
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.305741
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.420526
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.238480
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.291288
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.077949
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.159653
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.235081
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.180206
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.126309
Train Epoch: 1 [12800/60000 (